In [1]:
lst=[]
for i in range(10000):
  lst.append([str(i),"name"+str(i)])

In [2]:
in_data=spark.sparkContext.parallelize(lst)

In [3]:
in_df=spark.createDataFrame(in_data,["id","name"])

In [4]:
in_df.rdd.getNumPartitions()

Out[5]: 8

In [5]:
in_df.coalesce(1).write.mode("overwrite").saveAsTable("unbucket_demo1")

In [6]:
in_df.coalesce(1).write.bucketBy(4,"id").sortBy("id").mode("overwrite").saveAsTable("bucket_demo1")

In [7]:
df_read1=spark.table("unbucket_demo1")
df_read1.where("id='1000'").explain()

== Physical Plan ==
*(1) Project [id#1275, name#1276]
+- *(1) Filter (isnotnull(id#1275) AND (id#1275 = 1000))
 +- *(1) ColumnarToRow
 +- FileScan parquet default.unbucket_demo1[id#1275,name#1276] Batched: true, DataFilters: [isnotnull(id#1275), (id#1275 = 1000)], Format: Parquet, Location: InMemoryFileIndex[dbfs:/user/hive/warehouse/unbucket_demo1], PartitionFilters: [], PushedFilters: [IsNotNull(id), EqualTo(id,1000)], ReadSchema: struct<id:string,name:string>

In [8]:
df_read=spark.table("bucket_demo1")
df_read.where("id='1000'").explain()

== Physical Plan ==
*(1) Project [id#1288, name#1289]
+- *(1) Filter (isnotnull(id#1288) AND (id#1288 = 1000))
 +- *(1) ColumnarToRow
 +- FileScan parquet default.bucket_demo1[id#1288,name#1289] Batched: true, DataFilters: [isnotnull(id#1288), (id#1288 = 1000)], Format: Parquet, Location: InMemoryFileIndex[dbfs:/user/hive/warehouse/bucket_demo1], PartitionFilters: [], PushedFilters: [IsNotNull(id), EqualTo(id,1000)], ReadSchema: struct<id:string,name:string>, SelectedBucketsCount: 1 out of 4

In [9]:
spark.sql("Describe Extended bucket_demo1").show()

+--------------------+--------------------+-------+
 col_name| data_type|comment|
+--------------------+--------------------+-------+
 id| string| null|
 name| string| null|
 | | |
# Detailed Table ...| | |
 Database| default| |
 Table| bucket_demo1| |
 Owner| root| |
 Created Time|Sat Jul 25 08:21:...| |
 Last Access| UNKNOWN| |
 Created By| Spark 3.0.0| |
 Type| MANAGED| |
 Provider| parquet| |
 Num Buckets| 4| |
 Bucket Columns| [`id`]| |
 Sort Columns| [`id`]| |
 Location|dbfs:/user/hive/w...| |
 Serde Library|org.apache.hadoop...| |
 InputFormat|org.apache.hadoop...| |
 OutputFormat|org.apache.hadoop...| |
+--------------------+--------------------+-------+